## Searching for specific claims that AUPRC is suprior to AUROC with GPT 3.5/4

### Defining prompt and importing function

In [4]:
SYSTEM_PROMPT = """
You are an expert in machine learning and scientific literature review.
For each chunk of a published paper (which may have typos, misspellings, and odd characters as a result of conversion from PDF), return a JSON object that states whether or not the paper makes any claim that the area under the precision recall curve (AUPRC) is superior or inferior as a general performance metric to the area under the receiver operating characteristic (AUROC) in an ML setting, in particular for imbalanced classification problems. A paper claiming that a model performs better under AUPRC vs. AUROC is *not* an example of this; instead a paper claiming that AUPRC should be used instead of AUROC in cases of class imbalance is an example of this metric commentary. Respond with format {"claims": [{"claim": DESCRIPTION OF CLAIM, "evidence_quote": SUBSTRING FROM INPUT STATING CLAIM}, ...]}. If the paper makes no claims, leave the "claims" key in the JSON object empty. If the claim made is that the AUPRC is superior to the AUROC in the case of class imbalance, use the string "AUPRC is superior to AUROC for imbalanced data" for the description of the claim. For other claims, use any appropriate free-text description.

Examples: 

Input: "AUROC: The horizontal and vertical coordinates of the Receiver Operating Characteristic (ROC) curve are the FPR and TPR, and the curve is obtained by calculating the FPR and TPR under multiple sets of thresholds. The area of the region enclosed by the ROC curve and the horizontal axis is often used to evaluate binary classification tasks, denoted as AUROC. The value of AUROC is within the range of [0, 1], and higher values indicate better performance. AUROC can visualize the generalization performance of the GVAED model and help to select the best alarm threshold In addition, the Equal Error Rate (EER), i.e., the proportion of incorrectly classified frames when TPR and FNR are equal, is also used to measure the performance of anomaly detection models. AP: Due to the highly unbalanced nature of positive and negative samples in GVAED tasks, i.e., the TN is usually larger than the TP, researchers think that the area under the Precision-Recall (PR) curve is more suitable for evaluating GVAED models, denoted as AP. The horizontal coordinates of the PR curve are the Recall (i.e., the TPR in Eq. 4), while the vertical coordinate represents the Precision, defined as Precision = TP TP+FP . A point on the PR curve corresponds to the Precision and Recall values at a certain threshold."
Output: {"claims": [{"claim": "AUPRC is superior to AUROC for imbalanced data", "evidence_quote": "Due to the highly unbalanced nature of positive and negative samples in GVAED tasks, i.e., the TN is usually larger than the TP, researchers think that the area under the Precision-Recall (PR) curve is more suitable for evaluating GVAED models, denoted as AP"}]}

Input: "As seen, it outperforms other approaches except in the cases of TinyImageNet for CIFAR-100. Our approach still has better AUROC, but the detection error and FPR at 95% TPR are slightly larger than ODIN’s. Interestingly, the MD approach is worse than max-softmax in some cases. Such a result has also been reporte"
Output: {"claims": []}

Input: "AUC-ROC measures the class separability at various threshold settings. ROC is the probability curve and AUC represents the degree of measures of separability. It compares true positive rate (sensitivity/recall) versus the false positive rate (1 - specificity). The higher the AUC-ROC, the bigger the distinction between the true positive and false negative. • AUC-PR: It combines the precision and recall, for various threshold values, it compares the positively predicted value (precision) vs the true positive rate (recall). Both precision and recall focus on the positive class (the lesion) and unconcerned about the true negative (not a lesion, which is the majority class). Thus, for class imbalance, PR is more suitable than ROC. The higher the AUC-PR, the better the model performance"
Output: {"claims": [{"claim": "AUPRC is superior to AUROC for imbalanced data", "evidence_quote": "Thus, for class imbalance, PR is more suitable than ROC"}]}

So please for each chunk of the input, return a JSON object that states whether or not the paper makes any claim that the area under the precision recall curve (AUPRC) is superior or inferior as a general performance metric to the area under the receiver operating characteristic (AUROC) in an ML setting, in particular for imbalanced classification problems. 
"""

### Load df with context windows

In [ ]:
import pandas as pd
df_with_context_windows = pd.read_csv('data/filtered_data_with_context_windows_v4.csv')

### Setup environment 

In [ ]:
# Setting up the environment
import sys
import os
sys.path.append('src')  # Adjust this path to ensure it points to the correct directory
from claim_search_v3 import process_all_context_windows

# Set your OpenAI API key and other relevant parameters
model = "gpt-3.5-turbo-1106"
openai_api_key = "INSERT API KEY HERE"

### Process the context windows

In [5]:
# Process the DataFrame
df_claims = process_all_context_windows(df_with_context_windows, 
                                        model, 
                                        SYSTEM_PROMPT, 
                                        openai_api_key, 
                                        texts_before_pause=1000, 
                                        pause_duration=0.5, 
                                        max_workers = 6)

# Save the updated DataFrame
df_claims.to_csv('data/processed_gpt_responses_total_run_v2.csv', index=False)

Processed 1000/29498 texts; pausing for 0.5 seconds...
Processed 2000/29498 texts; pausing for 0.5 seconds...
Processed 3000/29498 texts; pausing for 0.5 seconds...
Processed 4000/29498 texts; pausing for 0.5 seconds...
Processed 5000/29498 texts; pausing for 0.5 seconds...
Processed 6000/29498 texts; pausing for 0.5 seconds...
Processed 7000/29498 texts; pausing for 0.5 seconds...
Processed 8000/29498 texts; pausing for 0.5 seconds...
Processed 9000/29498 texts; pausing for 0.5 seconds...
Processed 10000/29498 texts; pausing for 0.5 seconds...
Processed 11000/29498 texts; pausing for 0.5 seconds...
Processed 12000/29498 texts; pausing for 0.5 seconds...
Processed 13000/29498 texts; pausing for 0.5 seconds...
Processed 14000/29498 texts; pausing for 0.5 seconds...
Processed 15000/29498 texts; pausing for 0.5 seconds...
Processed 16000/29498 texts; pausing for 0.5 seconds...
Processed 17000/29498 texts; pausing for 0.5 seconds...
Processed 18000/29498 texts; pausing for 0.5 seconds...
P

### Extract the claims from the json format in the column "gpt_response"

In [9]:
import pandas as pd
import json

# Define function to extract claims
def extract_claim(row):
    try:
        data = json.loads(row)
        claims_for_row = []
        if 'claims' in data:  # Check if 'claims' key is in the dictionary
            for claim_dict in data['claims']:  # Iterate over each claim
                if isinstance(claim_dict, dict) and 'claim' in claim_dict:  # Check if it's a dictionary and has 'claim'
                    claims_for_row.append(claim_dict['claim'])
        return " | ".join(claims_for_row)
    except json.JSONDecodeError:
        return None  # Return None or an empty string '' if preferred
    except TypeError:
        return None  # Handle cases where the row might not be properly formatted

# Apply the function to each row in the 'gpt_response' column to create the new 'claim' column
df_filtered_claims_2['claim'] = df_filtered_claims_2['gpt_response'].apply(extract_claim)

/var/folders/rm/48r5vtns6fz6xtmhbq04sd7d03b5vb/T/ipykernel_27820/2795003071.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_claims_2['claim'] = df_filtered_claims_2['gpt_response'].apply(extract_claim)


### Find unique claims

In [29]:
unique_claims_array = df_filtered_claims_2['claim'].unique()
unique_claims_df = pd.DataFrame(unique_claims_array, columns=['Unique Claims'])
unique_claims_df.to_csv('claims.csv', index=False, sep=',', quoting=csv.QUOTE_ALL)

### Filter out only the claims we are interested in

In [31]:
# List of claims we want to keep
claims_to_keep = [
    "AUC is a common but worse measure than AP",
    "AUPRC is more sensitive than AUC in skewed data like CVR prediction task",
    "AUPRC is superior to AUROC for imbalanced data",
    "PR-AUC is more robust in the face of imbalanced data",
    "AUPRC is more suitable than AUROC for evaluating models",
    "AUPRC is not dependent on the choice of a specific threshold"
]

# Filter out the DataFrame rows where 'claim' is not in the claims_to_keep list
df_filtered_claims_3 = df_filtered_claims_2[df_filtered_claims_2['claim'].isin(claims_to_keep)]


### Save the new filtered claims

In [52]:
df_filtered_claims_3.to_csv('data/filtered_claims_new_v4.csv', index=False)

## GPT 4 Search

### Redefining systemt prompt, that does not have the last message from before. This is because we add such a description in the user prompt

In [2]:
SYSTEM_PROMPT = """
You are an expert in machine learning and scientific literature review.
For each chunk of a published paper (which may have typos, misspellings, and odd characters as a result of conversion from PDF), return a JSON object that states whether or not the paper makes any claim that the area under the precision recall curve (AUPRC) is superior or inferior as a general performance metric to the area under the receiver operating characteristic (AUROC) in an ML setting, in particular for imbalanced classification problems. A paper claiming that a model performs better under AUPRC vs. AUROC is *not* an example of this; instead a paper claiming that AUPRC should be used instead of AUROC in cases of class imbalance is an example of this metric commentary. Respond with format {"claims": [{"claim": DESCRIPTION OF CLAIM, "evidence_quote": SUBSTRING FROM INPUT STATING CLAIM}, ...]}. If the paper makes no claims, leave the "claims" key in the JSON object empty. If the claim made is that the AUPRC is superior to the AUROC in the case of class imbalance, use the string "AUPRC is superior to AUROC for imbalanced data" for the description of the claim. For other claims, use any appropriate free-text description.

Examples: 

Input: "AUROC: The horizontal and vertical coordinates of the Receiver Operating Characteristic (ROC) curve are the FPR and TPR, and the curve is obtained by calculating the FPR and TPR under multiple sets of thresholds. The area of the region enclosed by the ROC curve and the horizontal axis is often used to evaluate binary classification tasks, denoted as AUROC. The value of AUROC is within the range of [0, 1], and higher values indicate better performance. AUROC can visualize the generalization performance of the GVAED model and help to select the best alarm threshold In addition, the Equal Error Rate (EER), i.e., the proportion of incorrectly classified frames when TPR and FNR are equal, is also used to measure the performance of anomaly detection models. AP: Due to the highly unbalanced nature of positive and negative samples in GVAED tasks, i.e., the TN is usually larger than the TP, researchers think that the area under the Precision-Recall (PR) curve is more suitable for evaluating GVAED models, denoted as AP. The horizontal coordinates of the PR curve are the Recall (i.e., the TPR in Eq. 4), while the vertical coordinate represents the Precision, defined as Precision = TP TP+FP . A point on the PR curve corresponds to the Precision and Recall values at a certain threshold."
Output: {"claims": [{"claim": "AUPRC is superior to AUROC for imbalanced data", "evidence_quote": "Due to the highly unbalanced nature of positive and negative samples in GVAED tasks, i.e., the TN is usually larger than the TP, researchers think that the area under the Precision-Recall (PR) curve is more suitable for evaluating GVAED models, denoted as AP"}]}

Input: "As seen, it outperforms other approaches except in the cases of TinyImageNet for CIFAR-100. Our approach still has better AUROC, but the detection error and FPR at 95% TPR are slightly larger than ODIN’s. Interestingly, the MD approach is worse than max-softmax in some cases. Such a result has also been reporte"
Output: {"claims": []}

Input: "AUC-ROC measures the class separability at various threshold settings. ROC is the probability curve and AUC represents the degree of measures of separability. It compares true positive rate (sensitivity/recall) versus the false positive rate (1 - specificity). The higher the AUC-ROC, the bigger the distinction between the true positive and false negative. • AUC-PR: It combines the precision and recall, for various threshold values, it compares the positively predicted value (precision) vs the true positive rate (recall). Both precision and recall focus on the positive class (the lesion) and unconcerned about the true negative (not a lesion, which is the majority class). Thus, for class imbalance, PR is more suitable than ROC. The higher the AUC-PR, the better the model performance"
Output: {"claims": [{"claim": "AUPRC is superior to AUROC for imbalanced data", "evidence_quote": "Thus, for class imbalance, PR is more suitable than ROC"}]}
"""



### Defining introduction and end statement to the user prompt

In [3]:
introduction_statement_prompt = """
Please carefully review the following text. We are specifically looking for claims where AUPRC is argued to be a superior metric to AUROC, especially in cases of class imbalance in machine learning applications. Any claim that discusses the preference of AUPRC over AUROC due to its effectiveness in such scenarios should be returned in the a JSON object. If no such claims are found, please leave the 'claims' key empty. Here is the text:
"""
end_statement_prompt = """
If you find any claim asserting the superiority of AUPRC over AUROC for imbalanced datasets, please provide your findings in a JSON object with the key 'claims'. Each claim should be a dictionary with 'claim' and 'evidence_quote' as keys, like this: {"claims": [{"claim": "DESCRIPTION OF CLAIM", "evidence_quote": "SUBSTRING FROM INPUT STATING CLAIM"}]}. If no relevant claims are found, the 'claims' key should have an empty list.
"""

### Setting up environment

In [ ]:
import sys
import os
sys.path.append('src')  # Adjust this path to ensure it points to the correct directory

from claim_search_v4 import process_all_context_windows

model = "gpt-4-0125-preview"
openai_api_key = "INSERT API KEY HERE"

In [5]:
# Running the first 2000 rows, to make sure new setup is efficient
df_4_0_first_2000 = df_filtered_claims_3.iloc[:2000]
df_claims_4_0_first_2000 = process_all_context_windows(df_4_0_first_2000, model, SYSTEM_PROMPT, introduction_statement_prompt, end_statement_prompt, openai_api_key, texts_before_pause=500, pause_duration=0.5, max_workers=6)

# Save the updated DataFrame
df_claims_4_0_first_2000.to_csv('data/processed_gpt_4_0_responses_total_run_v2_first_2000.csv', index=False)

Processed 500/2000 texts; pausing for 0.5 seconds...
Processed 1000/2000 texts; pausing for 0.5 seconds...
Processed 1500/2000 texts; pausing for 0.5 seconds...
Processed 2000/2000 texts; pausing for 0.5 seconds...


### Works as intended, as such we are running the rest

In [7]:
df_4_0_rest = df_filtered_claims_3.iloc[2000:]
df_claims_4_0_rest = process_all_context_windows(df_4_0_rest, model, SYSTEM_PROMPT, introduction_statement_prompt, end_statement_prompt, openai_api_key, texts_before_pause=500, pause_duration=0.5, max_workers=6)

# Save the updated DataFrame
df_claims_4_0_rest.to_csv('data/processed_gpt_4_0_responses_total_run_v2_rest.csv', index=False)

Processed 500/7591 texts; pausing for 0.5 seconds...
Processed 1000/7591 texts; pausing for 0.5 seconds...
Processed 1500/7591 texts; pausing for 0.5 seconds...
Processed 2000/7591 texts; pausing for 0.5 seconds...
Processed 2500/7591 texts; pausing for 0.5 seconds...
Processed 3000/7591 texts; pausing for 0.5 seconds...
Processed 3500/7591 texts; pausing for 0.5 seconds...
Processed 4000/7591 texts; pausing for 0.5 seconds...
Processed 4500/7591 texts; pausing for 0.5 seconds...
Processed 5000/7591 texts; pausing for 0.5 seconds...
Processed 5500/7591 texts; pausing for 0.5 seconds...
Processed 6000/7591 texts; pausing for 0.5 seconds...
Processed 6500/7591 texts; pausing for 0.5 seconds...
Processed 7000/7591 texts; pausing for 0.5 seconds...
Processed 7500/7591 texts; pausing for 0.5 seconds...


### Combining the two dataframes

In [11]:
combined_df_4_0 = pd.concat([df_claims_4_0_first_2000, df_claims_4_0_rest])

### Extracting claims with previously defined function

In [12]:
# Apply the function to each row in the 'gpt_response' column to create the new 'claim' column
combined_df_4_0['claim'] = combined_df_4_0_no_non_clams['gpt_response'].apply(extract_claim)

### Saving a csv file with claims and the number of times they are used

In [23]:
import csv
unique_claims_array_4_0 = combined_df_4_0_no_non_clams['claim'].unique()
unique_claims_df_4_0 = pd.DataFrame(unique_claims_array_4_0, columns=['Unique Claims'])

# Counting occurrences of each claim
claim_counts = combined_df_4_0_no_non_clams['claim'].value_counts().reset_index()
claim_counts.columns = ['Unique Claims', 'Count']

# Merging
unique_claims_df_4_0_with_counts = pd.merge(unique_claims_df_4_0, claim_counts, on='Unique Claims', how='left')

# Saving to CSV
unique_claims_df_4_0_with_counts.to_csv('claims_4_0_with_counts.csv', index=False, sep=',', quoting=csv.QUOTE_ALL)

### Saving only relvevant claims

In [24]:
# List of claims we want to keep
claims_to_keep = [
"AUPRC is superior to AUROC for imbalanced data",
"AP may be considered a more attractive performance metric than AUC",
"AUPRC may be considered a more attractive performance metric than AUROC",
"AUPRC is superior to AUROC for imbalanced data | AUPRC is superior to AUROC for imbalanced data",
"AP is a better metric for discriminates the risk prediction performance than the AUC does",
"AUPRC might be preferred for datasets with known proportion of anomalies"
]

# Filter out the DataFrame rows where 'claim' is not in the claims_to_keep list
combined_df_4_0_filtered = combined_df_4_0[combined_df_4_0['claim'].isin(claims_to_keep)]


### Saving the final claims, so they can be manually annotated

In [26]:
combined_df_4_0_filtered.to_csv('data/filtered_claims_4_0_new_v4_final.csv', index=False)